In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q timm wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00


In [3]:
import os
import gc
import sys
import zipfile
from PIL import Image
import cv2
import math, random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from collections import OrderedDict

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

import timm
from transformers import get_cosine_schedule_with_warmup

import albumentations as A

from sklearn.model_selection import KFold

# import wandb
# wandb.login()

In [4]:
from_dir = '/content/drive/MyDrive/Kaggle/rsna-2024-lsdc-train/'
file_name = 'cvt_png.zip'
to_dir = '/content/rsna-2024-lsdc-train/'

!mkdir -p {to_dir}

!rsync -ah --progress {from_dir + file_name} {to_dir}

zip_path = os.path.join(to_dir, file_name)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    total_files = len(zip_ref.infolist())
    with tqdm(total=total_files, desc="Unzipping", unit="files") as pbar:
        for file in zip_ref.infolist():
            zip_ref.extract(file, to_dir)
            pbar.update(1)

!du -sh {to_dir}

sending incremental file list
cvt_png.zip
         10.58G 100%   32.61MB/s    0:05:09 (xfr#1, to-chk=0/1)


Unzipping: 100%|██████████| 86769/86769 [01:14<00:00, 1169.17files/s]


20G	/content/rsna-2024-lsdc-train/


In [5]:
# ORIGINAL DATASET PATH
# INPUT_PATH  = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'
INPUT_PATH  = '/content/drive/MyDrive/Kaggle/rsna-2024-lumbar-spine-degenerative-classification'

# PNG FILE PATH
# PNG_PATH = '/kaggle/input/rsna24lsdc-0815-eda-dcm2png' # Add Input -> Your Work + Notebook -> rsna24lsdc_0815_eda_dcm2png
PNG_PATH = '/content/rsna-2024-lsdc-train'

# MODEL OUTPUT PATH
# OUTPUT_PATH = '/kaggle/working'
OUTPUT_PATH = '/content/drive/MyDrive/Kaggle/rsna-2024-lsdc-train-split'

In [6]:
# IMG_SIZE = (512, 512)
IMG_SIZE = (224, 224)

IN_CHANS = 40
N_LABELS = 25
N_CLASSES = 3 * N_LABELS

AUG_PROB = 0.75

EPOCHS = 20
EARLY_STOPPING_EPOCH = 3

N_FOLDS = 5

MODEL_NAME = "edgenext_base.in21k_ft_in1k"          # 18M,  72MB
# MODEL_NAME = "tf_efficientnetv2_s"                  # 22M,  87MB
# MODEL_NAME = "convnext_tiny.fb_in22k_ft_in1k"       # 28M,  111MB
# MODEL_NAME = "convnextv2_tiny.fcmae_ft_in22k_in1k"  # 29M,  114MB
# MODEL_NAME = "convnextv2_small.fcmae_ft_in22k_in1k" # 50M,  200MB
# MODEL_NAME = "convnextv2_base.fcmae_ft_in22k_in1k"  # 89M,  356MB
# MODEL_NAME = "convnextv2_large.fcmae_ft_in22k_in1k" # 198M, 792MB
# MODEL_NAME = "swin_tiny_patch4_window7_224"         # 28M,  110MB
# MODEL_NAME = "swin_small_patch4_window7_224"        # 50M,  196MB
# MODEL_NAME = "swin_base_patch4_window7_224"         # 88M,  345MB
# MODEL_NAME = "swin_large_patch4_window7_224"        # 197M, 772MB

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

N_WORKERS = os.cpu_count()
print(N_WORKERS)

USE_AMP = True # Automatic Mixed Precision - True if using T4 or newer

BATCH_SIZE = 32
# BATCH_SIZE = 128

LR = 2e-4
MIN_LR = 1e-6
WD = 1e-2

SEED = 8620
def set_random_seed(seed: int = 8620, deterministic: bool = False):
    """Set seeds"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = deterministic  # type: ignore
set_random_seed(SEED)

12


In [7]:
df = pd.read_csv(f'{INPUT_PATH}/train.csv')
df.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild


In [8]:
df = df.fillna(-100)
label2id = {'Normal/Mild': 0, 'Moderate':1, 'Severe':2}
df = df.replace(label2id)
df.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,4646740,0,0,1,2,0,0,0,0,1,...,0,0,0,2,0,0,1,1,1,0
2,7143189,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8785691,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,10728036,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# 예외처리 할 study_id들 제거

print(len(df))

study_ids_to_remove = [3008676218, 2780132468, 2492114990]
df = df[~df['study_id'].isin(study_ids_to_remove)]

print(len(df))

1975
1972


In [10]:
MODEL_CONFIGS = {
    'sagittal_t1': {'in_chans': 10, 'n_labels': 10, 'start_channel': 0},
    'sagittal_t2_stir': {'in_chans': 10, 'n_labels': 5, 'start_channel': 10},
    'axial_t2': {'in_chans': 20, 'n_labels': 10, 'start_channel': 20}
}

class RSNA24Dataset(Dataset):
    def __init__(self, df, phase='train', transform=None, model_type='sagittal_t1'):
        self.df = df
        self.transform = transform
        self.phase = phase
        self.model_type = model_type
        self.config = MODEL_CONFIGS[model_type]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = np.zeros((512, 512, self.config['in_chans']), dtype=np.uint8)
        t = self.df.iloc[idx]
        st_id = int(t['study_id'])

        # Select labels based on the model type
        if self.model_type == 'sagittal_t1':
            label_cols = [col for col in self.df.columns if 'neural_foraminal_narrowing' in col]
        elif self.model_type == 'sagittal_t2_stir':
            label_cols = [col for col in self.df.columns if 'spinal_canal_stenosis' in col]
        else:  # axial_t2
            label_cols = [col for col in self.df.columns if 'subarticular_stenosis' in col]

        label = t[label_cols].values.astype(np.int64)

        start_channel = self.config['start_channel']
        for i in range(self.config['in_chans']):
            try:
                if self.model_type == 'sagittal_t1':
                    img_path = f'{PNG_PATH}/cvt_png/{st_id}/Sagittal T1/{i+1:02d}.png'
                elif self.model_type == 'sagittal_t2_stir':
                    img_path = f'{PNG_PATH}/cvt_png/{st_id}/Sagittal T2_STIR/{i+1:02d}.png'
                else:  # axial_t2
                    img_path = f'{PNG_PATH}/cvt_png/{st_id}/Axial T2/{i+1:02d}.png'

                img = Image.open(img_path).convert('L')
                img = np.array(img)
                x[..., i] = img.astype(np.uint8)
            except:
                print(f'failed to load on {st_id}, {self.model_type}')

        if self.transform is not None:
            x = self.transform(image=x)['image']
        x = x.transpose(2, 0, 1)

        return x, label

In [11]:
transforms_train = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=AUG_PROB),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),
    A.OneOf([
        A.MotionBlur(blur_limit=3),
        A.MedianBlur(blur_limit=3),
        A.GaussianBlur(blur_limit=3),
        A.GaussNoise(var_limit=(3.0, 9.0)),
    ], p=AUG_PROB),
    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.0),
        A.ElasticTransform(alpha=3),
    ], p=AUG_PROB),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),
    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.CoarseDropout(max_holes=16, max_height=int(IMG_SIZE[0] * 0.125), max_width=int(IMG_SIZE[1] * 0.125),
                    min_holes=1, min_height=int(IMG_SIZE[0] * 0.03125), min_width=int(IMG_SIZE[1] * 0.03125),
                    p=AUG_PROB),
    A.Normalize(mean=0.5, std=0.5)
])

transforms_val = A.Compose([
    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.Normalize(mean=0.5, std=0.5)
])

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:193: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  self.__pydantic_validator__.validate_python(data, self_instance=self)


In [12]:
class ClassificationModule(nn.Module):
#     def __init__(self, in_channels, num_classes, hidden_dim=512):
#         super().__init__()
#         self.conv = nn.Conv2d(in_channels, hidden_dim, kernel_size=3, padding=1)
#         self.bn = nn.BatchNorm2d(hidden_dim)
#         self.pool = nn.AdaptiveAvgPool2d(1)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         x = torch.relu(self.bn(self.conv(x)))
#         x = self.pool(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc(x)
#         return x

    def __init__(self, in_channels, num_classes, hidden_dim=512):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, hidden_dim, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_dim)
        self.conv2 = nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(hidden_dim)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class RSNA24Model(nn.Module):
    def __init__(self, base_model_name, in_chans, num_classes, pretrained=True):
        super().__init__()
        self.base_model = timm.create_model(base_model_name, pretrained=pretrained, in_chans=in_chans, features_only=True)

        # Get the number of output features from the last layer of the base model
        with torch.no_grad():
            dummy_input = torch.randn(1, in_chans, IMG_SIZE[0], IMG_SIZE[1])
            features = self.base_model(dummy_input)
            in_features = features[-1].shape[1]

        self.classification_module = ClassificationModule(in_features, num_classes)

    def forward(self, x):
        features = self.base_model(x)
        output = self.classification_module(features[-1])
        return output.view(output.size(0), -1)

In [14]:
# Setup automatic mixed precision (AMP) for faster training and reduced memory usage
# autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.bfloat16) # for newer Ampere GPUs
autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half) # for T4 or newer GPUs

# Initialize gradient scaler for AMP
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP, init_scale=4096)


<ipython-input-14-03dd172529ea>:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half) # for T4 or newer GPUs
<ipython-input-14-03dd172529ea>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP, init_scale=4096)


In [20]:
# Setup automatic mixed precision (AMP) for faster training and reduced memory usage
# autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.bfloat16) # for newer Ampere GPUs
autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half) # for T4 or newer GPUs

# Initialize gradient scaler for AMP
scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP, init_scale=4096)

def train_model(model_type):
    model_config = MODEL_CONFIGS[model_type]

    skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(skf.split(range(len(df)))):
        print('#'*30)
        print(f'start fold {fold} for {model_type}')
        print('#'*30)

        df_train = df.iloc[trn_idx]
        df_valid = df.iloc[val_idx]

        train_ds = RSNA24Dataset(df_train, phase='train', transform=transforms_train, model_type=model_type)
        train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, drop_last=True, num_workers=N_WORKERS)

        valid_ds = RSNA24Dataset(df_valid, phase='valid', transform=transforms_val, model_type=model_type)
        valid_dl = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, drop_last=False, num_workers=N_WORKERS)

        model = RSNA24Model(MODEL_NAME, model_config['in_chans'], model_config['n_labels'] * 3, pretrained=True)
        model.to(device)

        # Setup optimizer
        optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)

        class CosineScheduleWithMinLR:
            def __init__(self, scheduler, min_lr):
                self.scheduler = scheduler
                self.min_lr = min_lr

            def step(self):
                # Perform the cosine scheduler step
                self.scheduler.step()
                # Enforce the minimum learning rate
                for param_group in self.scheduler.optimizer.param_groups:
                    param_group['lr'] = max(param_group['lr'], self.min_lr)

            def get_last_lr(self):
                # Return the last learning rate applied
                return [max(lr, self.min_lr) for lr in self.scheduler.get_last_lr()]

        # Setup learning rate scheduler with warmup
        warmup_steps = EPOCHS / 10 * len(train_dl)
        num_total_steps = EPOCHS * len(train_dl)
        num_cycles = 0.475

        # Get the default cosine schedule
        cosine_scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=num_total_steps,
            num_cycles=num_cycles
        )

        # Wrap the cosine scheduler with the custom scheduler
        scheduler = CosineScheduleWithMinLR(cosine_scheduler, MIN_LR)

        weights = torch.tensor([1.0, 2.0, 4.0]).to(device)
        criterion = nn.CrossEntropyLoss(weight=weights)
        criterion2 = nn.CrossEntropyLoss(weight=weights.cpu())

        best_loss = math.log(3)
        best_wll = math.log(3)
        es_step = 0

        for epoch in range(1, EPOCHS + 1):
            model.train()
            total_loss = 0
            for idx, (x, t) in enumerate(tqdm(train_dl)):
                x = x.to(device)
                t = t.to(device)

                optimizer.zero_grad()

                with autocast:
                    loss = 0
                    y = model(x)
                    for col in range(model_config['n_labels']):
                        pred = y[:, col*3:col*3+3]
                        gt = t[:, col]
                        loss = loss + criterion(pred, gt) / model_config['n_labels']

                    total_loss += loss.item()

                if not math.isfinite(loss):
                    print(f"Loss is {loss}, stopping training")
                    sys.exit(1)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                if scheduler is not None:
                    scheduler.step()

            train_loss = total_loss / len(train_dl)
            print(f'train_loss: {train_loss:.6f}')

            model.eval()
            total_loss = 0
            y_preds = []
            labels = []

            with torch.no_grad():
                for idx, (x, t) in enumerate(tqdm(valid_dl)):
                    x = x.to(device)
                    t = t.to(device)

                    with autocast:
                        loss = 0
                        y = model(x)
                        for col in range(model_config['n_labels']):
                            pred = y[:, col*3:col*3+3]
                            gt = t[:, col]
                            loss = loss + criterion(pred, gt) / model_config['n_labels']
                            y_preds.append(pred.float().cpu())
                            labels.append(gt.cpu())

                        total_loss += loss.item()

            val_loss = total_loss / len(valid_dl)
            y_preds = torch.cat(y_preds, dim=0)
            labels = torch.cat(labels)
            val_wll = criterion2(y_preds, labels)

            print(f'val_loss: {val_loss:.6f}, val_wll: {val_wll:.6f}')

            if val_loss < best_loss or val_wll < best_wll:
                es_step = 0
                if val_loss < best_loss:
                    print(f'epoch: {epoch}, best loss updated from {best_loss:.6f} to {val_loss:.6f}')
                    best_loss = val_loss

                output_dir = f'{OUTPUT_PATH}/rsna-2024-lsdc-trained-models/{model_type}'
                os.makedirs(output_dir, exist_ok=True)

                if val_wll < best_wll:
                    print(f'epoch: {epoch}, best wll_metric updated from {best_wll:.6f} to {val_wll:.6f}')
                    best_wll = val_wll
                    fname = os.path.join(output_dir, f'best_wll_model_fold-{fold}.pt')
                    torch.save(model.state_dict(), fname)
            else:
                es_step += 1
                if es_step >= EARLY_STOPPING_EPOCH:
                    print('early stopping')
                    break

    print(f"Training completed for {model_type}")

# Train models for each image type
for model_type in MODEL_CONFIGS.keys():
    train_model(model_type)

<ipython-input-20-824e6ec4469c>:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half) # for T4 or newer GPUs
<ipython-input-20-824e6ec4469c>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=USE_AMP, init_scale=4096)


##############################
start fold 0 for sagittal_t1
##############################


100%|██████████| 49/49 [00:33<00:00,  1.44it/s]


train_loss: 1.004290


100%|██████████| 13/13 [00:03<00:00,  3.83it/s]


...

val_loss: 0.826712, val_wll: 0.914414


100%|██████████| 49/49 [01:02<00:00,  1.28s/it]


train_loss: 0.704362


100%|██████████| 13/13 [00:06<00:00,  2.04it/s]

val_loss: 0.743874, val_wll: 0.810087
early stopping
Training completed for axial_t2


In [19]:
def calculate_cv_score():
    cv_scores = {}
    for model_type in MODEL_CONFIGS.keys():
        y_preds = []
        labels = []
        weights = torch.tensor([1.0, 2.0, 4.0], device=device, dtype=torch.float32)
        criterion2 = nn.CrossEntropyLoss(weight=weights)

        skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
        for fold, (_, val_idx) in enumerate(skf.split(range(len(df)))):
            df_valid = df.iloc[val_idx]
            valid_ds = RSNA24Dataset(df_valid, phase='valid', transform=transforms_val, model_type=model_type)
            valid_dl = DataLoader(valid_ds, batch_size=1, shuffle=False, pin_memory=True, drop_last=False, num_workers=N_WORKERS)

            model = RSNA24Model(MODEL_NAME, MODEL_CONFIGS[model_type]['in_chans'], MODEL_CONFIGS[model_type]['n_labels'] * 3, pretrained=False)
            fname = f'{OUTPUT_PATH}/rsna-2024-lsdc-trained-models/{model_type}/best_wll_model_fold-{fold}.pt'
            model.load_state_dict(torch.load(fname, map_location=device, weights_only=True))
            model.to(device)

            model.eval()
            with torch.no_grad():
                for idx, (x, t) in enumerate(tqdm(valid_dl)):
                    x = x.to(device)
                    t = t.to(device)

                    with autocast:
                        y = model(x)
                        # Explicitly convert to float32
                        y = y.float()
                        for col in range(MODEL_CONFIGS[model_type]['n_labels']):
                            pred = y[:,col*3:col*3+3]
                            gt = t[:,col]
                            y_preds.append(pred.cpu())
                            labels.append(gt.cpu())

        y_preds = torch.cat(y_preds).float()
        labels = torch.cat(labels).long()

        # Move tensors to CPU for sklearn log_loss calculation
        y_preds_cpu = y_preds.cpu()
        labels_cpu = labels.cpu()

        cv = criterion2(y_preds.to(device), labels.to(device))
        print(f'CV score for {model_type}:', cv.item())
        cv_scores[model_type] = cv.item()

        # Calculate log loss
        from sklearn.metrics import log_loss
        y_pred_np = y_preds_cpu.softmax(1).numpy()
        labels_np = labels_cpu.numpy()
        y_pred2 = y_pred_np  # Remove the concatenation with y_pred_nan
        weights = []
        for l in labels_np:
            if l == 0: weights.append(1)
            elif l == 1: weights.append(2)
            elif l == 2: weights.append(4)
            else: weights.append(0)
        cv2 = log_loss(labels_np, y_pred2, normalize=True, sample_weight=weights, labels=[0, 1, 2])
        print(f'Log loss CV score for {model_type}:', cv2)
        cv_scores[f'{model_type}_log_loss'] = cv2

    return cv_scores

cv_scores = calculate_cv_score()
print("Final CV Scores:", cv_scores)

100%|██████████| 394/394 [00:07<00:00, 54.09it/s]


CV score for sagittal_t1: 0.6349097490310669
Log loss CV score for sagittal_t1: 0.6349098523953686


100%|██████████| 394/394 [00:07<00:00, 53.75it/s]


CV score for sagittal_t2_stir: 0.3826533854007721
Log loss CV score for sagittal_t2_stir: 0.38265343585701583


100%|██████████| 394/394 [00:09<00:00, 43.25it/s]

CV score for axial_t2: 0.7858816385269165
Log loss CV score for axial_t2: 0.7858815794562946
Final CV Scores: {'sagittal_t1': 0.6349097490310669, 'sagittal_t1_log_loss': 0.6349098523953686, 'sagittal_t2_stir': 0.3826533854007721, 'sagittal_t2_stir_log_loss': 0.38265343585701583, 'axial_t2': 0.7858816385269165, 'axial_t2_log_loss': 0.7858815794562946}


In [23]:
def calculate_combined_cv_score():
    y_preds_all = []
    labels_all = []
    weights = torch.tensor([1.0, 2.0, 4.0]).to(device)
    criterion2 = nn.CrossEntropyLoss(weight=weights)

    skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    for fold, (trn_idx, val_idx) in enumerate(skf.split(range(len(df)))):
        print('#'*30)
        print(f'start fold {fold}')
        print('#'*30)
        df_valid = df.iloc[val_idx]

        for model_type, config in MODEL_CONFIGS.items():
            valid_ds = RSNA24Dataset(df_valid, phase='valid', transform=transforms_val, model_type=model_type)
            valid_dl = DataLoader(valid_ds, batch_size=1, shuffle=False, pin_memory=True, drop_last=False, num_workers=N_WORKERS)

            model = RSNA24Model(MODEL_NAME, config['in_chans'], config['n_labels'] * 3, pretrained=False)
            fname = f'{OUTPUT_PATH}/rsna-2024-lsdc-trained-models/{model_type}/best_wll_model_fold-{fold}.pt'
            model.load_state_dict(torch.load(fname, map_location=device, weights_only=True))
            model.to(device)

            model.eval()
            with torch.no_grad():
                for idx, (x, t) in enumerate(tqdm(valid_dl, desc=f'Evaluating {model_type}')):
                    x = x.to(device)
                    t = t.to(device)

                    with autocast:
                        y = model(x)
                        y = y.float()  # Ensure float32
                        y_preds_all.append(y.view(-1, 3).cpu())
                        labels_all.append(t.view(-1).cpu())

    # Combine all predictions and labels
    y_preds_all = torch.cat(y_preds_all)
    labels_all = torch.cat(labels_all)

    # Calculate combined CV score
    cv = criterion2(y_preds_all.to(device), labels_all.to(device))
    print('Combined CV score:', cv.item())

    # Calculate log loss
    from sklearn.metrics import log_loss
    y_pred_np = y_preds_all.softmax(1).numpy()
    labels_np = labels_all.numpy()

    weights = []
    for l in labels_np:
        if l == 0: weights.append(1)
        elif l == 1: weights.append(2)
        elif l == 2: weights.append(4)
        else: weights.append(0)

    cv2 = log_loss(labels_np, y_pred_np, normalize=True, sample_weight=weights, labels=[0, 1, 2])
    print('Combined Log Loss CV score:', cv2)

    return cv.item(), cv2

# Calculate combined CV score
combined_cv, combined_log_loss = calculate_combined_cv_score()
print(f"Final Combined CV Score: {combined_cv}")
print(f"Final Combined Log Loss: {combined_log_loss}")

##############################
start fold 0
##############################


Evaluating axial_t2: 100%|██████████| 395/395 [00:08<00:00, 44.56it/s]


##############################
start fold 1
##############################


Evaluating axial_t2: 100%|██████████| 395/395 [00:08<00:00, 45.09it/s]


##############################
start fold 2
##############################


...

##############################
start fold 3
##############################


Evaluating axial_t2: 100%|██████████| 394/394 [00:08<00:00, 45.70it/s]


##############################
start fold 4
##############################


Evaluating axial_t2: 100%|██████████| 394/394 [00:08<00:00, 44.14it/s]


Combined CV score: 0.6539226770401001
Combined Log Loss CV score: 0.6539227384811536
Final Combined CV Score: 0.6539226770401001
Final Combined Log Loss: 0.6539227384811536
